In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark import SparkConf
import gc


print("loaded libraries")
spark = SparkSession.builder \
        .appName("worst labs") \
        .getOrCreate()

loaded libraries


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/20 16:08:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [97]:
labs = spark.read.parquet("/project2/wparker/SIPA_data/RCLIF_labs.parquet")
labs = labs.withColumn('lab_result_time',f.to_timestamp('lab_order_time','yyyy-MM-dd HH:mm:ss'))
labs = labs.select('C19_HAR_ID', 'lab_result_time','lab_name', 'lab_value')

select_expr = [f.regexp_replace(f.col('lab_name'), "[\ufeff]", "").alias('lab_name')]
labs = labs.select('C19_HAR_ID', 'lab_result_time', 'lab_value', *select_expr)

select_expr = [f.regexp_replace(f.col('lab_value'), "[\ufeff]", "").alias('lab_value')]
labs = labs.select('C19_HAR_ID', 'lab_result_time', 'lab_name', *select_expr)

In [95]:
cohort = spark.read.option("header",True).csv('/project2/wparker/SIPA_data/life_support_cohort.csv')
cohort = cohort.withColumn('life_support_start_time',f.to_timestamp('life_support_start','yyyy-MM-dd HH:mm:ss'))
cohort = cohort.select('C19_HAR_ID', 'life_support_start_time')

In [98]:
cohort_labs = cohort.join(labs,'C19_HAR_ID','left')
cohort_labs = cohort_labs.withColumn("hour_diff", (f.col("lab_result_time").cast("long")-f.col("life_support_start_time").cast("long"))/(60*60))
cohort_labs_48 = cohort_labs.filter((f.col('hour_diff')>-42)&(f.col('hour_diff')<=5))

In [ ]:
#test = cohort_labs_48.where(f.col('lab_value').like("%<%") | f.col('lab_value').like("%>%"))
#group_cols = ["lab_name", "lab_value"]
#test = test.where(~f.col('lab_value').like("% %"))
#test = test.groupBy(group_cols).count().orderBy("lab_name", "count").show(n=100, truncate=False)

In [99]:
cohort_labs_48 = cohort_labs_48.withColumn('meas_hour', f.hour(f.col('lab_result_time')))
cohort_labs_48 = cohort_labs_48.withColumn('meas_date', f.to_date(f.col('lab_result_time')))
cohort_labs_48 = cohort_labs_48.select('C19_HAR_ID', 'life_support_start_time','meas_date',
                                           'meas_hour', 'lab_name', 'lab_value')
cohort_labs_48 = cohort_labs_48.withColumn("lab_value_num",cohort_labs_48.lab_value.cast('double'))

In [107]:
group_cols = ["C19_HAR_ID", "life_support_start_time","meas_date", "meas_hour"]
cohort_labs_wide = cohort_labs_48.groupBy(group_cols) \
                                     .pivot("lab_name") \
                                     .agg(f.min('lab_value_num').alias("min"),
                                         f.max('lab_value_num').alias("max"))
cohort_labs_wide.write.parquet("/project2/wparker/SIPA_data/cohort_labs_48.parquet", mode="overwrite")

In [106]:
group_cols = ["C19_HAR_ID", "life_support_start_time"]
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
cohort_labs_wide_summary = cohort_labs_48.groupBy(group_cols) \
                                     .pivot("lab_name") \
                                     .agg(f.min('lab_value_num').alias("min"),
                                         f.max('lab_value_num').alias("max")).orderBy("life_support_start_time")

rows = cohort_labs_wide_summary.count()
print(f"DataFrame Rows count : {rows}")

cohort_labs_wide_summary.write.parquet("/project2/wparker/SIPA_data/cohort_labs_48_summary.parquet", mode="overwrite")

DataFrame Rows count : 10747
